# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo SINASC_RO_2019.csv` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo `estrutura_sinasc_para_CD.pdf` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
`
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis `['ESTCIVMAE', 'CONSULTAS']` possuem o código `9`, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código `9`.<br>
6. Substitua os valores faltantes da quantitativa (`QTDFILVIVO`) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção `inplace = True`.

In [39]:
import pandas as pd
import requests

# 1) seu código aqui
sinasc = pd.read_csv('SINASC_RO_2019.csv')
print(sinasc.shape)
sinasc.drop_duplicates().shape
# Não há duplicados

(27028, 69)


(27028, 69)

In [41]:
# 2) seu código aqui
print('Número de valores missing por variável:')
print(sinasc.isna().sum())

print('\nNúmero total de valores missing:')
print(sinasc.isna().sum().sum())

Número de valores missing por variável:
ORIGEM          0
CODESTAB      115
CODMUNNASC      0
LOCNASC         0
IDADEMAE        0
             ... 
munResUf        0
munResLat       1
munResLon       1
munResAlt       1
munResArea      1
Length: 69, dtype: int64

Número total de valores missing:
121594


In [81]:
# 3) seu código aqui
sinasc_interesse_estudo = sinasc[['LOCNASC', 
                           'IDADEMAE', 
                           'ESTCIVMAE', 
                           'ESCMAE', 
                           'QTDFILVIVO', 
                           'GESTACAO', 
                           'GRAVIDEZ', 
                           'CONSULTAS', 
                           'APGAR5']].copy()

print('Contagem de missing por variável:')
print(sinasc_interesse_estudo.isna().sum())

print('\nContagem de todos os missing:')
print(sinasc_interesse_estudo.isna().sum().sum())

Contagem de missing por variável:
LOCNASC          0
IDADEMAE         0
ESTCIVMAE      317
ESCMAE         312
QTDFILVIVO    1573
GESTACAO      1232
GRAVIDEZ        79
CONSULTAS        0
APGAR5         103
dtype: int64

Contagem de todos os missing:
3616


In [83]:
# 4) seu código aqui
print('Total de linhas:')
print(len(sinasc_interesse_estudo))
print("Dados faltantes na coluna 'APGAR5':")
print(sinasc_interesse_estudo['APGAR5'].isna().sum())
print()

sinasc_interesse_estudo.dropna(subset=['APGAR5'], inplace=True)

print('Linhas após a remoção:')
print(len(sinasc_interesse_estudo))
print("Dados faltantes na coluna 'APGAR5' após a remoção:")
print(sinasc_interesse_estudo['APGAR5'].isna().sum())

Total de linhas:
27028
Dados faltantes na coluna 'APGAR5':
103

Linhas após a remoção:
26925
Dados faltantes na coluna 'APGAR5' após a remoção:
0


In [85]:
# 5) seu código aqui
print('Contagem de dados faltantes por variável:')
print(sinasc_interesse_estudo.isna().sum())

sinasc_interesse_estudo.fillna(value={'ESTCIVMAE': 9, 
                               'CONSULTAS': 9}, 
                        inplace=True)

print('\nContagem de dados faltantes por variável após considerar dados faltantes como código 9:')
print(sinasc_interesse_estudo.isna().sum())

Contagem de dados faltantes por variável:
LOCNASC          0
IDADEMAE         0
ESTCIVMAE      315
ESCMAE         310
QTDFILVIVO    1566
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64

Contagem de dados faltantes por variável após considerar dados faltantes como código 9:
LOCNASC          0
IDADEMAE         0
ESTCIVMAE        0
ESCMAE         310
QTDFILVIVO    1566
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64


In [87]:
# 6) Seu código aqui
print("Dados faltantes na coluna 'QTDFILVIVO':")
print(sinasc_interesse_estudo['QTDFILVIVO'].isna().sum())

sinasc_interesse_estudo['QTDFILVIVO'] = sinasc_interesse_estudo['QTDFILVIVO'].fillna(value=0)

print("\nDados faltantes na coluna 'QTDFILVIVO' após a substituição:")
print(sinasc_interesse_estudo['QTDFILVIVO'].isna().sum())

Dados faltantes na coluna 'QTDFILVIVO':
1566

Dados faltantes na coluna 'QTDFILVIVO' após a substituição:
0


In [89]:
# 7) seu código aqui
print('Valores missing por variável:')
print(sinasc_interesse_estudo.isna().sum())

sinasc_interesse_estudo.fillna(value={'ESCMAE':9, 
                               'GESTACAO': 9, 
                               'GRAVIDEZ': 9}, 
                        inplace=True)

print('\nContagem de valores missing por variável após substituição:')
print(sinasc_interesse_estudo.isna().sum())

Valores missing por variável:
LOCNASC          0
IDADEMAE         0
ESTCIVMAE        0
ESCMAE         310
QTDFILVIVO       0
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64

Contagem de valores missing por variável após substituição:
LOCNASC       0
IDADEMAE      0
ESTCIVMAE     0
ESCMAE        0
QTDFILVIVO    0
GESTACAO      0
GRAVIDEZ      0
CONSULTAS     0
APGAR5        0
dtype: int64


In [91]:
# 8) seu código aqui
sinasc_interesse_estudo.loc[(sinasc_interesse_estudo['APGAR5'] >= 8) & 
                     (sinasc_interesse_estudo['APGAR5'] <= 10), 
                     'APGAR5_CLASSIFICADO'] = 'normal'

sinasc_interesse_estudo.loc[(sinasc_interesse_estudo['APGAR5'] >= 6) & 
                     (sinasc_interesse_estudo['APGAR5'] <= 7), 
                     'APGAR5_CLASSIFICADO'] = 'asfixia leve'

sinasc_interesse_estudo.loc[(sinasc_interesse_estudo['APGAR5'] >= 4) & 
                     (sinasc_interesse_estudo['APGAR5'] <= 5), 
                     'APGAR5_CLASSIFICADO'] = 'asfixia moderada'

sinasc_interesse_estudo.loc[sinasc_interesse_estudo['APGAR5'] <= 3, 
                     'APGAR5_CLASSIFICADO'] = 'asfixia severa'

print('Frequências por grau de asfixia do bebê:')
print(sinasc_interesse_estudo['APGAR5_CLASSIFICADO'].value_counts())

print('\nFrequências por grau de asfixia do bebê em porcentagem:')
print(round(sinasc_interesse_estudo['APGAR5_CLASSIFICADO'].value_counts() / len(sinasc_interesse_estudo) * 100, 2))

Frequências por grau de asfixia do bebê:
APGAR5_CLASSIFICADO
normal              26463
asfixia leve          320
asfixia severa         74
asfixia moderada       68
Name: count, dtype: int64

Frequências por grau de asfixia do bebê em porcentagem:
APGAR5_CLASSIFICADO
normal              98.28
asfixia leve         1.19
asfixia severa       0.27
asfixia moderada     0.25
Name: count, dtype: float64


In [93]:
# 9) seu código aqui
sinasc_interesse_estudo.rename(columns={'LOCNASC': 'local_de_nascimento', 
                                 'IDADEMAE': 'idade_da_mae', 
                                 'ESTCIVMAE': 'estado_civil_da_mae', 
                                 'ESCMAE': 'escolaridade_da_mae__anos_de_estudo_concluídos', 
                                 'QTDFILVIVO': 'quantidade_de_filhos_vivos', 
                                 'GESTACAO': 'semanas_de_gestacao', 
                                 'GRAVIDEZ': 'tipo_de_gravidez', 
                                 'CONSULTAS': 'numero_de_consultas_de_prenatal', 
                                 'APGAR5': 'apgar_no_quinto_minuto', 
                                 'APGAR5_CLASSIFICADO': 'apgar5_classificado'},
                       inplace=True)

In [95]:
sinasc_interesse_estudo

,local_de_nascimento,idade_da_mae,estado_civil_da_mae,escolaridade_da_mae__anos_de_estudo_concluídos,quantidade_de_filhos_vivos,semanas_de_gestacao,tipo_de_gravidez,numero_de_consultas_de_prenatal,apgar_no_quinto_minuto,apgar5_classificado
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,normal
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0,normal
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0,normal
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0,normal
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0,normal
...,...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0,normal
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0,normal
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,normal
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0,normal
